추천을 위한 알고리즘을 분류해보면 크게 메모리 기반과 모델 기반으로 나눌 수 있다.

메모리 기반 추천 : 추천을 위한 데이터를 모두 메모리에 가지고 있으면서 추천이 필요할 때마다 이 데이터를 사용해서 계산을 해서 추천하는 방식  
모델 기반 추천 : 데이터로부터 추천을 위한 모델을 구성한 후 이 모델만 저장하고, 실제 추천을 할 때에는 이 모델을 사용해서 추천을 하는 방식

메모리 기반 추천은 원래 데이터를 충실하게 사용하는 장점이 있지만 대량의 데이터를 다뤄야 하는 상용 사이트에서 계신시간이 너무 오래걸린다는 단점이 있다.
<br/>
모델 기반 추천 방식은 데이터는 모형을 만드는 데만 사용하고 모델이 만들어지면 원래 데이터는 사용하지 않기 때문에 대규모 사용 사이트에서 필요한 빠른 반응이 가능하지만 모델을 만드는 과정에서 많은 계산이 필요하다.

일반적으로 메모리 기반 추천은 개별 사용자의 데이터에 집중하고, 모델 기반 추천은 전체 사용자의 평가 패턴으로부터 모델을 구성하기 때문에 데이터가 가지고 있는 약한 신호도 더 잘 잡아내는 장점이 있다.
<br/>
이번 장에서는 대표적인 모델 기반 추천 알고리즘인 MF 방식 추천에 대해 알아보기로 한다.

### 4.1 Matrix Factorization 방식의 원리

(사용자 X 아이템) 으로 구성된 하나의 행렬을 2개의 행렬로 분해하는 방법이다.  
앞서 CF 를 구현할 때 사용한 full matrix 는 (사용자 X 아이템) 형태의 2차원 행렬이다.  
이를 사용자 잠재요인행렬과 아이템 잠재요인행렬로 나눌 수 있다.

<img src = 'https://velog.velcdn.com/images/sangyun/post/07d390cb-b5b2-4ece-a095-b60a2805af8a/image.png' width = 600 height = 300>

R 행렬을 사용자행렬P와 아이템행렬Q로 쪼개어 분석하는 것이 MF 방식이다.  
P, Q 행렬에서 공통인 K개의 요인을 잠재요인으로 부르는데 사용자와 아이템의 특성을 K개의 잠재요인을 사용해서 분석하는 것이 MF 방식의 원리이다.

### 4.2 SGD를 사용한 MF 알고리즘

MF의 핵심은 주어진 사용자, 아이템의 관계를 가장 잘 설명하는 P, Q 행렬을 분해하는 것이다.  
주어진 (사용자X아이템) 평점 행렬인 R로부터 P,Q를 분해하는 알고리즘을 개념적으로 설명하면 다음과 같다.
1. 잠재요인의 개수 K를 정한다. K는 경험에 의한 직관적으로 정해도 되고, 다양한 K를 비교하면서 최적의 수를 정해도 된다.
2. 주어진 K에 따라 P, Q 행렬을 만들고 초기화한다.
3. 주어진 P,Q 행렬을 사용해 예측 평점을 구한다.
4. 실제 평점과 예측 평점을 비교해서 오차를 구하고, 이 오차를 줄이기 위해 P,Q 를 수정한다.
5. 전체 오차가 미리 정해진 기준값 이하가 되거나 미리 정해진 반복 횟수에 도달할 떄까지 3으로 돌아가서 반복한다.

여기서 핵심은 4에서 예측 오차를 줄이기 위해 어떻게 수정하는가이다.  
가장 일반적인 방법은 기계학습에서 많이 사용되는 SGD 방법을 적용하는 것이다.

SGD 방법을 적용하는 자세한 내용은 아래 링크 참조하자.  
https://bladejun.tistory.com/57

### 4.3 SGD를 사용한 MF 기본 알고리즘

In [6]:
import numpy as np
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

In [9]:
# MF class
class MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    def train(self): 
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # SGD for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            training_process.append((i+1, rmse))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f " % (i+1, rmse))
        return training_process

    # Rating prediction for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # SGD to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])


# 전체 데이터 사용 MF
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
mf = MF(R_temp, K=30, alpha=0.01, beta=0.02, iterations=100, verbose=True)
train_process = mf.train()

Iteration: 10 ; Train RMSE = 0.8841 
Iteration: 20 ; Train RMSE = 0.7098 
Iteration: 30 ; Train RMSE = 0.5946 
Iteration: 40 ; Train RMSE = 0.5424 
Iteration: 50 ; Train RMSE = 0.5149 
Iteration: 60 ; Train RMSE = 0.4979 
Iteration: 70 ; Train RMSE = 0.4867 
Iteration: 80 ; Train RMSE = 0.4786 
Iteration: 90 ; Train RMSE = 0.4727 
Iteration: 100 ; Train RMSE = 0.4679 


결과가 좋게 나오는데 train/test set 을 나누지 않으므로 당연한 결과라고 할 수 있다.

### 4.4 train test 분리 MF 알고리즘

3장의 CF 와 마찬가지로 train/test set 분리하여 훈련한다.  
차이점은 3장의 CF는 sklearn의 train_test_split 을 사용했는데 여기서는 sklearn 의 shuffle을 사용한다.  
3장에서는 층화추출을 할 수 있지만 여기서는 무작위로 섞기 때문에 극단적인 경우 특정 사용자의 모든 데이터가 train이나 test set 한 곳에 들어갈 수 있다.  
어떤 것을 사용할 지는 데이터분석의 목적에 따라서 선택하면 된다.

In [11]:
# train test 분리
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=42)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

In [12]:
# New MF class for training & testing
class NEW_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose = True):
        self.R = np.array(ratings)
        # user_id, item_id를 R의 index 와 매핑하기 위한 dictionary 생성
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_id_item = dict(index_item_id)
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)

        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # train set 의 RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x,y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x,y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))
        
    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    # SGD to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    
    # Test set 을 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)): # test 데이터에 있는 각 데이터에 대해서
            x = self.user_id_index[ratings_test.iloc[i, 0]]
            y = self.item_id_index[ratings_test.iloc[i, 1]]
            z = ratings_test.iloc[i, 2]
            test_set.append([x,y,z])
            self.R[x,y] = 0 # setting test set ratings to 0
        self.test_set = test_set
        return test_set # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error / len(self.test_set))
    
    # Training 하면서 test set 의 정확도를 계산
    def test(self):
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])

    # Full user-movie rating matrix
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)

In [13]:
# Testing MF RMSE
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
mf = NEW_MF(R_temp, K=30, alpha=0.001, beta=0.02, iterations=100, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration: 10 ; Train RMSE = 0.9683 ; Test RMSE = 0.9779
Iteration: 20 ; Train RMSE = 0.9437 ; Test RMSE = 0.9573
Iteration: 30 ; Train RMSE = 0.9326 ; Test RMSE = 0.9488
Iteration: 40 ; Train RMSE = 0.9258 ; Test RMSE = 0.9442
Iteration: 50 ; Train RMSE = 0.9210 ; Test RMSE = 0.9413
Iteration: 60 ; Train RMSE = 0.9171 ; Test RMSE = 0.9394
Iteration: 70 ; Train RMSE = 0.9133 ; Test RMSE = 0.9380
Iteration: 80 ; Train RMSE = 0.9091 ; Test RMSE = 0.9367
Iteration: 90 ; Train RMSE = 0.9041 ; Test RMSE = 0.9353
Iteration: 100 ; Train RMSE = 0.8976 ; Test RMSE = 0.9336


In [16]:
# Printing predictions
print(mf.full_prediction())
print()
print(mf.get_one_prediction(1, 2))

[[3.85564441 3.32316216 3.12375302 ... 3.33004042 3.46481574 3.39712718]
 [4.004832   3.44816665 3.17469406 ... 3.44104715 3.56375196 3.5300982 ]
 [3.26955944 2.73155049 2.44012766 ... 2.72011118 2.79986137 2.78649544]
 ...
 [4.11931287 3.55905619 3.27808885 ... 3.55337875 3.66567676 3.64270499]
 [4.33558492 3.75979083 3.50586094 ... 3.75328923 3.86902823 3.83429048]
 [3.8246876  3.28891336 3.0275446  ... 3.26907413 3.38101899 3.3507281 ]]

3.323162161517802
